<a href="https://colab.research.google.com/github/ArjunRAj77/ICD10-code-extractor/blob/main/ICD10_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ICD 10 Extractor

The aim of the following machine learning model is to create a model that will identify the corresponding ICD10 code of a symptom/disease.

Installing required packages:

In [ ]:
!pip install transformers[sentencepiece] datasets

In [ ]:
pip install transformers

In [2]:
pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import list_datasets
list_datasets()

In [6]:
from sklearn.model_selection import train_test_split

Loading the data from the CSV file. 

It contains list of disease and its corresponding ICD10 code.

In [7]:
import pandas as pd

icdcodedf = pd.read_csv('/content/ICD10.csv')
features=["ICDCode","Disease"]
icd10 = icdcodedf[features]
diseaselist = icd10['Disease'].tolist()
icdlist = icd10['ICDCode'].tolist()

icdcodedf

,ICDCode,Disease,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,A00.0,Cholera due to Vibrio cholerae 01,NaN,NaN,NaN
1,A00.1,Cholera due to Vibrio cholerae 01,NaN,NaN,NaN
2,A00.9,Cholera,NaN,NaN,NaN
3,A01.00,Typhoid fever,NaN,NaN,NaN
4,A01.1,Paratyphoid fever A,NaN,NaN,NaN
...,...,...,...,...,...
14810,C90.11,Plasma cell leukemia in remission,NaN,NaN,NaN
14811,C90.12,Plasma cell leukemia in relapse,NaN,NaN,NaN
14812,C88.8,Other malignant immunoproliferative diseases,NaN,NaN,NaN
14813,C90.21,Extramedullary plasmacytoma in remission,NaN,NaN,NaN


Cleaning the above dataset into a labelled dataset with string values.

In [8]:
icdfinallist=[]
diseasefinallist=[]
datalen=len(icd10)
for i in range(datalen):
  str_X = str(icdlist[i])
  str_Y = str(diseaselist[i])
  icdfinallist.append(str_X)
  diseasefinallist.append(str_Y)

new_df=zip(icdfinallist,diseasefinallist)
column_names = ['ICDCode', 'Disease']
df = pd.DataFrame(new_df,columns = column_names)

df

,ICDCode,Disease
0,A00.0,Cholera due to Vibrio cholerae 01
1,A00.1,Cholera due to Vibrio cholerae 01
2,A00.9,Cholera
3,A01.00,Typhoid fever
4,A01.1,Paratyphoid fever A
...,...,...
14810,C90.11,Plasma cell leukemia in remission
14811,C90.12,Plasma cell leukemia in relapse
14812,C88.8,Other malignant immunoproliferative diseases
14813,C90.21,Extramedullary plasmacytoma in remission


Converting the above dataframe into a new CSV file.

In [9]:
df.to_csv('/content/new_ICD10.csv')

*make sure that the entire dataset values are in string format, otherwise the following codes will not work.*

Loading a custom dataset in python:


Reference: https://huggingface.co/docs/datasets/v2.7.1/en/package_reference/loading_methods#datasets.load_dataset

In [10]:
from datasets import load_dataset
ICDData = load_dataset('csv', data_files='/content/new_ICD10.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cb076edab4707611/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
ICDData

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 14815
    })
})

In [12]:
ICDData['train'][0]

{'Unnamed: 0': 0,
 'ICDCode': 'A00.0',
 'Disease': 'Cholera due to Vibrio cholerae 01'}

In [13]:
ICDData['train'] = ICDData['train'].shuffle(seed=1).select(range(4000))
ICDData['train']

Dataset({
    features: ['Unnamed: 0', 'ICDCode', 'Disease'],
    num_rows: 4000
})

In [21]:
ICDData_train_validation = ICDData['train'].train_test_split(train_size=0.8)
ICDData_train_validation

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 2560
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 640
    })
})

In [23]:
ICDData['test'] = ICDData_train_validation['test'].shuffle(seed=1).select(range(600))
ICDData['test']

Dataset({
    features: ['Unnamed: 0', 'ICDCode', 'Disease'],
    num_rows: 600
})

In [24]:
ICDData_train_validation['test']

Dataset({
    features: ['Unnamed: 0', 'ICDCode', 'Disease'],
    num_rows: 640
})

In [25]:
ICDData_train_validation['validation'] = ICDData_train_validation.pop('test')
ICDData_train_validation

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 2560
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 640
    })
})

In [26]:
ICDData.update(ICDData_train_validation)
ICDData

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 2560
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 640
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 600
    })
})

In [29]:
from transformers import AutoTokenizer
# checkpoint = "distilbert-base-cased"
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(batch):
    return tokenizer(batch["Disease"], padding=True, truncation=True)

ICDData_encoded = ICDData.map(tokenize_function, batched=True, batch_size=None)

ICDData_encoded

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

TypeError: ignored

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(checkpoint, num_labels=num_labels)
         .to(device))

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(ICDData_encoded["train"]) // batch_size
model_name = f"{checkpoint}-icdCode-encoder"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )
training_args

In [ ]:
import transformers
import re

[x for x in dir(transformers) if re.search(r'^AutoModel', x)]

In [ ]:
ICDData_encoded

In [ ]:
ICDData_encoded["train"]

In [ ]:
ICDData_encoded["validation"]

In [ ]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(model=model, 
                  args=training_args, 
                  train_dataset=ICDData_encoded["train"],
                  eval_dataset=ICDData_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

In [ ]:
preds = trainer.predict(ICDData_encoded['validation'])
preds

In [ ]:
preds.predictions.shape

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))

In [ ]:
def get_accuracy(preds):
  predictions = preds.predictions.argmax(axis=-1)
  labels = preds.label_ids
  accuracy = accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))
  return {'accuracy': accuracy}

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [63]:
model_name

'distilbert-base-cased-icdCode-encoder'

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model_name)
classifier('Hepatitis A with hepatic coma')

In [ ]:
classifier('Hepatitis A without hepatic coma')